Najpierw import potrzebnych bibliotek i podpięcie pod dysk google

In [40]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

from pathlib import Path
!pip install langchain sentence-transformers faiss-cpu pypdf transformers torch langchain-community
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.schema import Document
import re

Mounted at /content/drive


Linki do tych artykułów:
- Google Scholar od razu pdf: Znaczenie rozwoju technologii dla konkurencyjności pasażerskiego transportu lotniczego
- Google Scholar od razu pdf: ZMIANY W KONSTRUKCJACH I WYPOSAŻENIU CYWILNYCH STATKÓW PASAŻERSKICH W KONTEKŚCIE ZWIĘKSZENIA BEZPIECZEŃSTWA
- https://open.icm.edu.pl/items/e8bdd2ee-8e78-4de4-8a54-fe18b6cc1f66
- https://yadda.icm.edu.pl/baztech/element/bwmeta1.element.baztech-article-BPK6-0020-0008

Teraz wczytanie artykułów na temat samolotów z dysku i podział na fragmenty

In [41]:
file_paths = [
    '/content/drive/MyDrive/Koleje_kontra_samoloty.pdf',
    '/content/drive/MyDrive/bezpieczenstwo_przewozu_www.pdf',
    '/content/drive/MyDrive/Zmiany_w_konstrukcjach_i_wyposażeni.pdf',
    '/content/drive/MyDrive/Znaczenie_rozwoju_technologii_dla_k.pdf'
]

all_docs = []
for file_path in file_paths:
    file = Path(file_path)
    if file.is_file():
        try:
            loader = PyPDFLoader(file_path)
            all_docs.extend(loader.load())
        except Exception as e:
            print(f"Błąd podczas ładowania pliku {file_path}: {e}")
    else:
        print(f"Plik nie znaleziony: {file_path}")


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)
docs = text_splitter.split_documents(all_docs)

Stworzenie bazy wektorowej

In [42]:
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
embeddings = HuggingFaceEmbeddings(model_name=model_name)
db = FAISS.from_documents(docs, embeddings)
db.save_local("faiss_index_hf")

Inicjalizacja modelu QA i generowanie odpowiedzi oraz inicjalizacja historii czatu wraz z pamięcią konwersacji i jej swobodą

In [ ]:
db = FAISS.load_local(
    "faiss_index_hf",
    embeddings,
    allow_dangerous_deserialization=True
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key='answer'
)

model_name = "sshleifer/distilbart-cnn-6-6"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

text_generator = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=350,
    do_sample=True
)

llm = HuggingFacePipeline(pipeline=text_generator)

custom_prompt = PromptTemplate(
    template="""Answer the question based on context and conversation history.
Be conversational, diverse, and avoid repeating previous answers.

Context: {context}

Conversation history: {chat_history}

Question: {question}

Answer:""",
    input_variables=["context", "chat_history", "question"]
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": custom_prompt},
    return_source_documents=True,
    verbose=False
)

print("Type 'exit/quit/bye/goodbye' to exit\n")

while True:
    query = input("Write your question: ").strip()

    if query.lower() in ['exit', 'quit', 'bye', 'goodbye']:
        break
    if not query:
        print("Ask a question. Don't leave it blank!")
        continue

    try:
        result = qa_chain.invoke({"question": query})
        print(f"\nAI Assistant: {result['answer']}\n")

    except Exception as e:
        print(f"Error: {e}")

Device set to use cpu


Type 'exit/quit/bye/goodbye' to exit

Write your question: Why airplanes matters?

AI Assistant: The civil aviation has more than a hundred years of history . The aviation industry is an innovative industry sector and is a seedbed of progress in thought to other industries . The question is about the benefits and challenges of uncrewed cargo and passenger transport as well as single-pilot flights, focusing on technological, operational

Write your question: Monkey likes dog?

AI Assistant:  The aviation industry is an innovative industry sector and is a seedbed of progress in thought to other industries . The question is about the benefits and challenges of uncrewed cargo and passenger transport as well as single-pilot flights, focusing on technological, operational and operational aspects .



Wnioski:
- udało się zrobić asystena AI
- dobrze, że można wpisywać pytania a on na nie odpowiada
 - dużo można zmieniać parametrów i bawić się nimi
Jedyne co, to siedziałem nad tym długo i czas już goni, ale nie wiem czemu na inne pytanie nie chce odpowiadać inaczej niż w ten sam sposób oraz coś nie tak z wykrywaniem, że pytanie nie na temat - na dniach u siebie to spróbuję zrobić, ale działą :)